In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [7]:
# TAVILY_API_KEY = ""
import os
os.environ["TAVILY_API_KEY"] = ""

In [8]:
search = TavilySearchResults()

In [9]:
search.invoke("what is the weather in Hyderabad")

[{'url': 'https://www.timeanddate.com/weather/india/hyderabad/ext',
  'content': 'Hyderabad 14 Day Extended Forecast. Time/General. Weather. Time Zone. DST Changes. Sun & Moon. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 83 °F. Clear.'},
 {'url': 'https://www.accuweather.com/en/in/hyderabad/202190/weather-forecast/202190',
  'content': 'Hyderabad, Telangana, India Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'},
 {'url': 'https://www.timeanddate.com/weather/india/hyderabad',
  'content': 'Current weather in Hyderabad and forecast for today, tomorrow, and next 14 days'},
 {'url': 'https://www.bbc.com/weather/1269843',
  'content': 'Hyderabad - Weather warnings issued 14-day forecast. Weather warnings issued. Forecast - Hyderabad. Day by day forecast. Last updated today at 08:31. Today, Sunny and a gentle breeze.'},
 {'url': 'https://www.bbc.com/weather/1176734',
  'content': '14

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
OPENAI_API_KEY = ""

In [13]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

In [15]:
retriever = vector.as_retriever()
retriever.get_relevant_documents("how to upload dataset")

[Document(page_content="to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn how to create an account, obtain an API key, and configure your environment.Pricing: Learn about the pricing model for LangSmith.Self-Hosting: Learn about self-hosting options for LangSmith.Proxy: Learn about the proxy capabilities of LangSmith.Tracing: Learn about the tracing capabilities of LangSmith.Evaluation: Learn about the evaluation capabilities of LangSmith.Prompt Hub Learn about the Prompt Hub, a prompt management tool built into LangSmith.Additional Resources‚ÄãLangSmith Cookbook: A collection of tutorials and end-to-end walkthroughs using LangSmith.LangChain Python: Docs for the Python LangChain library.LangChain Python API Reference: documentation to review the core APIs of LangChain.LangChain JS: Docs for the TypeScript LangChain libraryDiscord: Join us on our Discord to discuss all things LangChain!Cont

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [18]:
tools = [search,retriever_tool]

In [19]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [21]:
!pip install langchainhub

  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.31.64 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.2.1 which is incompatible.
unstructured 0.12.6 requires urllib3==1.26.18, but you have urllib3 2.2.1 which is incompatible.


In [22]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(model, tools, prompt)

In [24]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [26]:
agent_executor.invoke({"input": "what is the temp in hyderabd, Telangana"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'temperature in Hyderabad, Telangana'}`


[{'url': 'https://www.timeanddate.com/weather/india/hyderabad/ext', 'content': 'Hyderabad 14 Day Extended Forecast. Time/General. Weather. Time Zone. DST Changes. Sun & Moon. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 83 °F. Clear.'}, {'url': 'https://www.bbc.com/weather/1269843', 'content': '14-day weather forecast for Hyderabad.'}, {'url': 'https://www.accuweather.com/en/in/hyderabad/202190/weather-forecast/202190', 'content': 'Hyderabad, Telangana, India Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'}, {'url': 'https://www.accuweather.com/en/in/hyderabad/202190/current-weather/202190', 'content': 'Current weather in Hyderabad, Telangana, India. Check current conditions in Hyderabad, Telangana, India with radar, hourly, and more.'}, {'url'

{'input': 'what is the temp in hyderabd, Telangana',
 'output': 'The current temperature in Hyderabad, Telangana is 83°F and it is clear. You can find more detailed weather information on [Time and Date](https://www.timeanddate.com/weather/india/hyderabad/ext) or [AccuWeather](https://www.accuweather.com/en/in/hyderabad/202190/current-weather/202190).'}

In [27]:
agent_executor.invoke({"input": "how to test langchain"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how to test LangChain'}`


Skip to main contentü¶úÔ∏èüõ†Ô∏è LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookLangSmithOn this pageLangSmithIntroduction‚ÄãLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start‚ÄãTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps‚ÄãCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith sup

{'input': 'how to test langchain',
 'output': 'To test LangChain, you can follow these steps:\n\n1. **Tracing**: Get started with the tracing quick start guide.\n2. **Evaluation**: Get started with the evaluation quick start guide.\n\nAdditionally, you can check out the following sections to learn more about LangSmith:\n- **User Guide**: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.\n- **Setup**: Learn how to create an account, obtain an API key, and configure your environment.\n- **Pricing**: Learn about the pricing model for LangSmith.\n- **Self-Hosting**: Learn about self-hosting options for LangSmith.\n- **Proxy**: Learn about the proxy capabilities of LangSmith.\n- **Prompt Hub**: Learn about the Prompt Hub, a prompt management tool built into LangSmith.\n\nYou can also explore the LangSmith Cookbook, which is a collection of tutorials and end-to-end walkthroughs using LangSmith.\n\nIf you have any specific questions or need further a

In [28]:
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is Praveen Reddy Chinnareddy"),
            AIMessage(content="Hello Praveen Reddy Chinnareddy! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Praveen Reddy Chinnareddy.

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is Praveen Reddy Chinnareddy'),
  AIMessage(content='Hello Praveen Reddy Chinnareddy! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Praveen Reddy Chinnareddy.'}